### Fase 1: Extracción de Datos de API de MoviesDataset


En esta fase se utilizará la API de MoviesDataset para extraer información relevante sobre películas y cortometrajes.

**Requerimientos:**
- Películas desde 1990 hasta la actualidad.
- Géneros: Drama, Comedy, Action, Fantasy, Horror, Mystery, Romance, Thriller.
- Información necesaria:
  - Tipo (corto o película).
  - Nombre.
  - Año y mes de estreno.
  - ID de la película.


_______________________

In [125]:
import pandas as pd
import pickle
import requests
from tqdm import tqdm
import os
import dotenv
dotenv.load_dotenv()

True

In [126]:
url = "https://moviesdatabase.p.rapidapi.com/titles"
api_key = os.getenv("api_key")

lista_generos = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]
lista_tipos = ["movie", "short"]
lista_final = []

for genero in lista_generos:
    print(f"Buscando películas del género {genero}...")
    for anio_peli in tqdm(range(1990, 2025)):
        for tipo_peli in lista_tipos:

            # Llamada a la API
            querystring = {"genre": genero,
                        "titleType": tipo_peli,
                        "year": str(anio_peli),
                        "sort":"year.incr",
                        "limit":"50"}

            headers = {
                "x-rapidapi-key": api_key,
                "x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
            }

            response = requests.get(url, headers=headers, params=querystring)
            res = response.json()

            # Obtenemos los datos deseados y los almacenamos en una lista
            for i in range(len(res["results"])):
                id = res["results"][i]["id"]
                titulo = res["results"][i]["originalTitleText"]["text"]
                tipo = res["results"][i]["titleType"]["text"]
                anio = res["results"][i]["releaseYear"]["year"]
                
                # Si no hay mes lo dejamos a none
                try:
                    mes = res["results"][i]["releaseDate"]["month"]
                except:
                    mes = None
                
                # Si no ha filtrado bien el tipo o el año no guardamos la peli
                if tipo.lower() != tipo_peli:
                    print(f"  El tipo es {tipo} cuando debería ser {tipo_peli}")
                    continue
                if anio != anio_peli:
                    print(f"  El año de la peli es {anio} cuando debería ser {anio_peli}")
                    continue
                
                lista_final.append((id, titulo, genero, tipo, anio, mes))
    
    # Cada vez que acaba un genero lo almacenamos en el pkl
    with open('../datos/datos_api.plk', 'wb') as f:
        pickle.dump(lista_final, f)


    

Buscando películas del género Drama...


100%|██████████| 35/35 [00:28<00:00,  1.21it/s]


Buscando películas del género Comedy...


100%|██████████| 35/35 [00:27<00:00,  1.29it/s]


Buscando películas del género Action...


...

100%|██████████| 35/35 [00:25<00:00,  1.37it/s]


Buscando películas del género Romance...


100%|██████████| 35/35 [00:27<00:00,  1.26it/s]


Buscando películas del género Thriller...


100%|██████████| 35/35 [00:55<00:00,  1.58s/it]


Cargamos el pickle que hemos guardado con todos los datos recogidos de la API y creamos un dataframe:

In [128]:
with open('../datos/datos_api.plk', 'rb') as f:
     data = pickle.load(f)

In [ ]:
df = pd.DataFrame(lista_final, columns =['id', 'title', 'genre', 'type', "release_year", "release_month"])
df.sample(5)

,id,title,genre,type,release_year,release_month
3679,tt0206829,The Honky Problem,Comedy,Short,1991,NaN
2793,tt10105354,Cascaritas,Drama,Short,2017,10.0
26976,tt27053740,A Christmas Gift,Thriller,Short,2024,1.0
26840,tt12494932,72,Thriller,Movie,2023,6.0
12098,tt1245913,"Vincent, le magnifique",Fantasy,Short,2008,3.0


In [136]:
df.to_pickle("../datos/df_pelis.pkl")